In [1]:
from scrapy.crawler import CrawlerRunner, CrawlerProcess
import scrapy
from crochet import setup, wait_for
import os
import json

In [2]:
urls = [
     s  'https://www.americanas.com.br/busca/celular?limit=56',
        'https://www.americanas.com.br/busca/televisao?limit=51'
       ]

In [3]:
class AmericanasSetSpider(scrapy.Spider):
    name = "americanas-produtos"

    file_links = []
    
    def start_requests(self):
        for i, link in enumerate(self.start_urls):
            name_file = link.split("/")[-1].split("?")[0]
            self.file_links.append(open(f"../data/{name_file}.json",'w'))
            self.file_links[i].write("{\"products\":[")
            request = scrapy.Request(link, callback=self.parse, cb_kwargs=dict(index_file=i))
            yield request
            
    def parse(self, response, index_file):
        
        for link_product in response.xpath('//div[re:test(@class, "inStockCard__Wrapper-sc-1ngt5zo-0 iRvjrG")]//a[1]/@href').getall():

            complete_link = f"https://www.americanas.com.br{link_product}"

            yield response.follow(complete_link, callback=self.parseproduct, cb_kwargs=dict(index_file=index_file))
            
 
    def parseproduct(self, response, index_file):
        title = response.xpath('//h1[re:test(@class, "product-title__Title-sc-1hlrxcw-0 jyetLr")]/text() ').get()
        title = title.replace("\"", "\\\"")
        description = response.xpath('//div[re:test(@class, "src__Description-sc-13f3i2j-2 bqnMru")]//text() ').getall()
        description = "".join(description).replace("\n", " ").replace("\"", "\\\"")
        self.file_links[index_file].write(f"{{\"title\":\"{title}\", \"description\":\"{description}\"}},")


In [4]:
SETTINGS = {
            'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',         
            } 

In [5]:
process = CrawlerProcess(SETTINGS)
process.crawl(AmericanasSetSpider,start_urls= urls)
process.start()

2021-11-08 23:29:49 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-11-08 23:29:49 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.3 (default, Jul  2 2020, 16:21:59) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 2.9.2, Platform Linux-5.11.0-38-generic-x86_64-with-glibc2.10
2021-11-08 23:29:49 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-11-08 23:29:49 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, '
               'like Gecko) Chrome/92.0.4515.131 Safari/537.36'}
2021-11-08 23:29:49 [scrapy.extensions.telnet] INFO: Telnet Password: cfa64b523a7a7da3
2021-11-08 23:29:49 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsag

2021-11-08 23:29:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/3130393068?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=60b6b1bf52131c3c814d502c&buyboxToken=ZWxlbWVuby1hY3Rvci12MS1hY29tLTEyNzdiNzZlLTk1ZWMtNDdiNy1hMDBiLWVmM2JjNWZhZGIyOQ==> (referer: https://www.americanas.com.br/busca/televisao?limit=70)
2021-11-08 23:29:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/3060874906?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=611a9c4609c351890dd03413&buyboxToken=ZWxlbWVuby1hY3Rvci12MS1hY29tLWZiYzcxOWExLTBmMDUtNDRjYS1iNzM2LTYzN2IwOWE5NDVlYQ==> (referer: https://www.americanas.com.br/busca/televisao?limit=70)
2021-11-08 23:29:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/3953974576?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=61649adc09c351890d273a8b> (referer: https://www.

2021-11-08 23:29:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/1618074407?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=5ed50dab79bf8430cb1cd021&buyboxToken=ZWxlbWVuby1hY3Rvci12MS1hY29tLTM4OTk1NGFkLTczNzUtNDE2Yi05YjcyLTliMmUyMTBmNWQzYQ==> (referer: https://www.americanas.com.br/busca/televisao?limit=70)
2021-11-08 23:29:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/1515649301?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=5fd0dc4e0c070442666f5409&buyboxToken=ZWxlbWVuby1hY3Rvci12MS1hY29tLThlY2Y5OWZiLWYwNjUtNGY1ZS1hZmEzLWUzMTEyMzJjOGVmNg==> (referer: https://www.americanas.com.br/busca/televisao?limit=70)
2021-11-08 23:29:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/1902703080?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=60ae917052131c3c81586360&buyboxToken=ZWxlbWVuby1

2021-11-08 23:29:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/134544943?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=6116ea54549cd5594f0eeed7&buyboxToken=ZWxlbWVuby1hY3Rvci12MS1hY29tLTc0Y2YyZjYzLWY3MjAtNDExMC04NjkzLTFjNjVkMzczMWU3Mg==> (referer: https://www.americanas.com.br/busca/televisao?limit=70)
2021-11-08 23:29:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/3069509622?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=611abab909c351890d096b87&buyboxToken=ZWxlbWVuby1hY3Rvci12MS1hY29tLTJiYzM0ZmE2LTk3OWUtNDgzZS04ODEzLTAzYWM5YjY3MGU2Ng==> (referer: https://www.americanas.com.br/busca/televisao?limit=70)
2021-11-08 23:29:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/3069506106?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=608acb2152131c3c81e23044&buyboxToken=ZWxlbWVuby1h

In [6]:
def json_correction(start_urls):
    for i, link in enumerate(start_urls):
            #import pdb;pdb.set_trace()
            name_file = link.split("/")[-1].split("?")[0]
            
            with open(f"../data/{name_file}.json", 'rb+') as filehandle:
                filehandle.seek(-1, os.SEEK_END)
                filehandle.truncate()
                filehandle.write(b"]}")

In [7]:
json_correction(urls)

In [8]:
json.load(open("../data/celular.json",'rb'))

{'products': [{'title': "Smartphone Motorola Moto G60s 128GB 4G Wi-Fi Tela 6,8'' Dual Chip 6GB RAM Câmera Quádrupla + Selfie 16MP - Azul",
   'description': 'O  Smartphone Motorola Moto G60s chegou para surpreender os apaixonados por tecnologia.   Conta com o design atraente e perfeito para você segurar com apenas uma mão. Sua tela é LCD IPS 6.8", ideal para assistir vídeos, jogar e acessar as suas redes sociais preferidas. Um modelo completo.  Outro ponto que chama a atenção é o seu processador Helio G95 (2,0 GHz Octa-Core), que permite que o smartphone seja ainda mais ágil e você possa aproveitar ao máximo. Conte com muito espaço para adicionar fotos, vídeos e muitos arquivos.   O apaixonados por fotos vão adorar a câmera quádrupla traseira de 64MP + 8MP + 5MP + 2MP , que contam com resolução incrível para tirar fotos tanto de dia como de noite. Já a câmera de Selfie vem com 16MP, aquela opção perfeita para fazer os vídeos para postar nas redes sociais.   Outro diferencial do Smartph